In [1]:
from utils.helper import read_xml, descompress_zip, count_ingredients
import os
import pandas as pd

In [2]:
data_path = os.path.join("..", "data")

In [3]:
coctails = read_xml(os.path.join(data_path, "ccc_cocktails.xml"))
print(len(coctails))
print(type(coctails))

108
<class 'xml.etree.ElementTree.Element'>


In [4]:
df = pd.read_csv(os.path.join(data_path, "recipes_w_search_terms.csv"))
n, ingredients = count_ingredients(df["ingredients"])
print(f"Ingredients: \n {ingredients}")
print(f"Number of ingredients: \n {n}")

Ingredients: 
 ['purple tomato', 'boysenberries', 'vegetable stock cubes', 'onion glaze', 'chocolate marshmallow ice cream', 'candy valentine heart', 'fresh orange juice', 'grape jelly', 'red delicious apple', 'chanterelle mushrooms', 'skinless dark meat chicken', 'frozen white corn', 'cod steaks', 'ground lemon myrtle', 'flaked sea salt', 'pub mustard', 'canned chilies', 'miniature kisses', 'hawaiian bread', 'russian salad dressing', 'simply potatoes mashed sweet potatoes', 'hershey candy cane kisses', 'progresso%26trade%3b recipe starters%26trade%3b fire roasted tomato cooking sauce', 'marshmallow peep', 'icing sugar', 'gluten-free chocolate chips', 'niter kebbeh', 'deer heart', 'skinless chicken thigh', 'frozen crabmeat', 'marjoram', 'ricotta cheese', 'skinless boneless turkey breasts', 'turkey legs', 'tuna packed in oil', 'mayhaw', 'white grapefruit', 'maitake mushroom', 'ground turmeric', 'fontina cheese', 'pecans', 'mushroom soup mix', 'apricot mango wasabi sauce', 'light canned 